# Trabajo Final: Análisis y Resúmenes Automáticos de Noticias usando GPT y Técnicas de Prompting

**Autor:** Ariel Agüero  
**Nombre del curso:** Inteligencia Artificial: Generación de Prompts  
**Comisión:** 67115  
**Nombre del proyecto:** Análisis y Resúmenes Automáticos de Noticias usando GPT y Técnicas de Prompting  

---

## Resumen
 
Este proyecto tiene como objetivo desarrollar un sistema que, a partir de noticias, pueda realizar resúmenes automáticos, clasificar sentimientos y generar análisis iterativos mediante técnicas avanzadas de prompting. El enfoque integra modelos GPT y herramientas específicas para la recuperación de información y análisis contextual.


Indice 

1. Introducción
2. Objetivos
3. Metodología
4. Herramientas y tecnología
5. Implementación
6. Resultados
7. Conclusiones
8. Referencias

1. Introducción

2. Objetivos

Crear un sistema que permita:

- **2.1** Resumir automáticamente artículos de noticias.
- **2.2** Clasificar el sentimiento del contenido.
- **2.3** Generar etiquetas relevantes. (Aún falta)
- **2.4** Hacer preguntas iterativas para profundizar en el análisis de una noticia. (Aún falta)


3. Metodología

- **3.1** One-shot prompting para la generación de resúmenes.
- **3.2** Few-shot prompting para clasificar sentimientos con ejemplos previos.
- **3.3** Prompt iterativo para afinar las consultas.
- **3.4** Función calling para interactuar con funciones predefinidas (como extraer nombres de personas, lugares o fechas).


4. Herramientas y tecnología

- **4.1** Python
- **4.2** OpenAI
- **4.3** NewsAPI

 

5. Implementación

5.1 Import openAI dependences (load with conda install openai)

In [6]:
#Imports 
import requests
import openai
from datetime import datetime, timedelta


5.2 Use the API Key

In [7]:
NEWS_API_KEY = ""
openai.api_key = "" #Example -> dw-bQyr9128F49VJRNKAnST32lbkFJ2Zv4qEhWo4R0Xh6yrh15

In [10]:
# Función para obtener noticias populares de Argentina
def obtener_noticias_argentina_populares(tema, numero_resultados=5):
    # Rango de los últimos 3 días
    fecha_hoy = datetime.now()
    fecha_inicio = (fecha_hoy - timedelta(days=3)).strftime('%Y-%m-%d')
    fecha_fin = fecha_hoy.strftime('%Y-%m-%d')
    
    # Endpoint de NewsAPI
    url = (
        f"https://newsapi.org/v2/everything?"
        f"q={tema} AND Argentina&"
        f"language=es&"
        f"from={fecha_inicio}&to={fecha_fin}&"
        f"sortBy=popularity&"
        f"pageSize={numero_resultados}&"
        f"apiKey={NEWS_API_KEY}"
    )
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        articulos = data.get("articles", [])
        return [(art["title"], art["description"], art["url"], art["publishedAt"]) for art in articulos]
    else:
        print(f"Error al obtener noticias: {response.status_code}")
        return []

# Función para procesar noticias con OpenAI
def procesar_noticias(articulos):
    resultados = []
    for titulo, descripcion, url, fecha in articulos:
        # Prompt para generar un resumen en español
        prompt_resumen = f"""
        Título: {titulo}
        Descripción: {descripcion}

        Por favor, genera un resumen conciso en español del contenido del artículo.
        """
        response_resumen = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt_resumen}],
            max_tokens=100
        )
        resumen = response_resumen.choices[0]['message']['content']
        
        # Prompt para clasificar el sentimiento en español
        prompt_sentimiento = f"""
        Título: {titulo}
        Resumen: {resumen}

        Clasifica el sentimiento del artículo como Positivo, Negativo o Neutral. Responde en español.
        """
        response_sentimiento = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt_sentimiento}],
            max_tokens=50
        )
        sentimiento = response_sentimiento.choices[0]['message']['content'].strip()
        
        resultados.append((titulo, resumen, sentimiento, url, fecha))
    return resultados

# Main: Integra búsqueda de noticias con procesamiento
if __name__ == "__main__":
    tema = input("Introduce el tema de interés: ")
    
    # Obtener noticias populares de Argentina en español
    noticias = obtener_noticias_argentina_populares(tema)
    if not noticias:
        print("No se encontraron noticias.")
    else:
        print("\nNoticias encontradas:")
        for idx, (titulo, _, url, fecha) in enumerate(noticias, 1):
            print(f"{idx}. {titulo} ({fecha}) ({url})")
        
        # Procesar noticias
        print("\nProcesando noticias con OpenAI...")
        resultados = procesar_noticias(noticias)
        
        # Mostrar resultados
        print("\nResultados:")
        for idx, (titulo, resumen, sentimiento, url, fecha) in enumerate(resultados, 1):
            print(f"{idx}. {titulo}")
            print(f"   Resumen: {resumen}")
            print(f"   Sentimiento: {sentimiento}")
            print(f"   Fecha de publicación: {fecha}")
            print(f"   URL: {url}")


Introduce el tema de interés:  Autos



Noticias encontradas:
1. Qué se siente manejar el Toyota Corolla Cross híbrido (2024-11-28T22:45:28Z) (https://www.lanacion.com.ar/autos/que-se-siente-manejar-el-toyota-corolla-cross-hibrido-nid28112024/)
2. Llegó un nuevo sedán a la Argentina: cómo es y qué ofrece (2024-11-28T16:32:18Z) (https://www.lanacion.com.ar/autos/llego-un-nuevo-sedan-a-la-argentina-como-es-y-que-ofrece-nid28112024/)
3. Las ventas de autos cero kilómetro cayeron 20,5% en noviembre (2024-11-29T22:44:35Z) (https://www.lanacion.com.ar/economia/las-ventas-de-autos-cero-kilometro-cayeron-205-en-noviembre-nid29112024/)
4. Tras tres meses de repunte, volvieron a caer las ventas de autos (2024-11-29T22:26:55Z) (https://www.lanacion.com.ar/autos/tras-tres-meses-de-repunte-volvieron-a-caer-las-ventas-de-autos-nid29112024/)
5. Al menos 75 inmuebles en el Centro Histórico son utilizados como bodegas por asiáticos (2024-11-29T08:25:53Z) (https://www.jornada.com.mx/2024/11/29/capital/027n2cap)

Procesando noticias con OpenA

## 6. Resultados

Cada resultado ahora contiene:

- **El título del artículo.**
- **Un resumen conciso generado por OpenAI.**
- **La clasificación del sentimiento.**
- **La URL del artículo.**